In [ ]:
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array
import os, shutil
import numpy as np
from skimage.transform import resize
#This program reads models from a folder (example: Those saved by Modelcheckpoint in tensorboard) and runs predictions on a
#folder(s) of images.
imageheight=200
imagewidth=300
cutoff=0.7 #Above which is an echolcation and below is abnormal file-Used for displaying the min. cutoff to calculate from
mfolder = "./weights/" #Folder with saved models in h5 format
ifolder = "../abcopy/a/" #Folder with images to test model with
foldercheck2 = True #Do you want to check the 2nd folder of images?
ifolder2 = "../images/train/echolocation/" #Folder2 with images to test model with
foldercheck3 = False #Do you want to check the 3rd folder of images?
ifolder3 = "../abcopy/a/" #Folder3 with images to test model with

modcount=0 #Counts the model number for display 
for mdl in range(25): #Remove unwanted models-Range value represents the last model-1 to delete
    if os.path.exists(mfolder+str(mdl)+".h5"):
        os.remove(mfolder+str(mdl)+".h5")
    if os.path.exists(mfolder+"0"+str(mdl)+".h5"):
        os.remove(mfolder+"0"+str(mdl)+".h5")   
print('Reading directories from model and image folders')
mfilelist = os.listdir(mfolder) #Make list of model filenames
ifilelist = os.listdir(ifolder) #Make list of image filenames
if foldercheck2 == True:
    ifilelist2 = os.listdir(ifolder2) #Make list of image filenames for 2nd scan
if foldercheck3 == True:
    ifilelist3 = os.listdir(ifolder3) #Make list of image filenames for 3rd scan
for ipremove in mfilelist: #Remove .ipynb_checkpoints folder which may get saved in models folder
    if '{}'.format(ipremove[0:6]) == '.ipynb':
        shutil.rmtree(mfolder+ipremove)
mfilelist = os.listdir(mfolder) #Make list of model filenames again if .ipynb_checkpoints existed before 
print('Processing {0:2d} images in {1:2d} models-------------------------'.format(len(ifilelist),len(mfilelist)))
cutoffdiff = int((1-cutoff)*10) #Calculates the difference between the cutoff and 1-Used for calculating & displaying accuracies
for mdl in range(len(mfilelist)):
    modcount += 1
    model = load_model(mfolder+mfilelist[mdl])
    #model.summary()
    abcount = [0] * cutoffdiff 
    for inum in range(len(ifilelist)):
        image = load_img(ifolder + ifilelist[inum], target_size=(imageheight,imagewidth))
        imageresize = image.resize((imagewidth, imageheight)) #Resize image for desired CNN model
        imagearray = img_to_array(imageresize) #Make image into an array
        imagearray = np.expand_dims(imagearray, 0) #Add 4th dimension to array for CNN -> will get error otherwise
        imagearray = imagearray.astype('float32')/255 #Convert array so that its predictions are readable
        prediction=model.predict(imagearray) #Run prediction on image-Returns list of float predictions
        for abdetermine in range(cutoffdiff):
            if prediction[0]<(cutoff+(abdetermine*0.1)):
                abcount[abdetermine]=abcount[abdetermine]+1
    print('{0:2d}. of {1:2d} Model: {2:s}'.format(modcount,len(mfilelist),mfilelist[mdl]))
    for abprint in range(cutoffdiff):
        print('Abnormal @ cutoff {0:2.1f}: {1:2d} at {2:4.2f}% Echolocation count: {3:2d} at {4:4.2f}%'.format(cutoff+(abprint*0.1),
        abcount[abprint],abcount[abprint]*100/len(ifilelist),(len(ifilelist)-abcount[abprint]),             
        (len(ifilelist)-abcount[abprint])*100/len(ifilelist)))
    if foldercheck2 == True:
        #print('Processing {0:2d} images in each model'.format(len(ifilelist2)))
        abcount = [0] * cutoffdiff
        for inum in range(len(ifilelist2)):
            image = load_img(ifolder2 + ifilelist2[inum], target_size=(imageheight,imagewidth))
            imageresize = image.resize((imagewidth, imageheight)) #Resize image for desired CNN model
            imagearray = img_to_array(imageresize) #Make image into an array
            imagearray = np.expand_dims(imagearray, 0) #Add 4th dimension to array for CNN -> will get error otherwise
            imagearray = imagearray.astype('float32')/255 #Convert array so that its predictions are readable
            prediction=model.predict(imagearray) #Run prediction on image-Returns list of float predictions
            for abdetermine in range(cutoffdiff):
                if prediction[0]<(cutoff+(abdetermine*0.1)):
                    abcount[abdetermine]=abcount[abdetermine]+1
        print('Folder 2 results:')
        for abprint in range(cutoffdiff):
            print('Abnormal @ cutoff {0:2.1f}: {1:2d} at {2:4.2f}% Echolocation count: {3:2d} at {4:4.2f}%'.format(cutoff+(abprint*0.1),
            abcount[abprint],abcount[abprint]*100/len(ifilelist2),(len(ifilelist2)-abcount[abprint]),             
            (len(ifilelist2)-abcount[abprint])*100/len(ifilelist2)))
    if foldercheck3 == True:
        #print('Processing {0:2d} images in each model'.format(len(ifilelist3)))
        abcount = [0] * cutoffdiff
        for inum in range(len(ifilelist3)):
            image = load_img(ifolder3 + ifilelist3[inum], target_size=(imageheight,imagewidth))
            imageresize = image.resize((imagewidth, imageheight)) #Resize image for desired CNN model
            imagearray = img_to_array(imageresize) #Make image into an array
            imagearray = np.expand_dims(imagearray, 0) #Add 4th dimension to array for CNN -> will get error otherwise
            imagearray = imagearray.astype('float32')/255 #Convert array so that its predictions are readable
            prediction=model.predict(imagearray) #Run prediction on image-Returns list of float predictions
            for abdetermine in range(cutoffdiff):
                if prediction[0]<(cutoff+(abdetermine*0.1)):
                    abcount[abdetermine]=abcount[abdetermine]+1
        print('Folder 3 results:')
        for abprint in range(cutoffdiff):
            print('Abnormal @ cutoff {0:2.1f}: {1:2d} at {2:4.2f}% Echolocation count: {3:2d} at {4:4.2f}%'.format(cutoff+(abprint*0.1),
            abcount[abprint],abcount[abprint]*100/len(ifilelist3),(len(ifilelist3)-abcount[abprint]),             
            (len(ifilelist3)-abcount[abprint])*100/len(ifilelist3)))
    print('End of model {0:s} predictions-----------------------'.format(mfilelist[mdl]))
print('Finished')